In [ ]:
%matplotlib inline
import matplotlib as mplt 
from matplotlib import cm 
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec
import matplotlib 
from matplotlib.ticker import FuncFormatter 
import matplotlib.pyplot as plt
# matplotlib.style.use('ggplot')

from utils_libs import *
# from utils_training import *

import numpy as np
import pickle

from scipy.stats import norm

In [ ]:
def softmax_stable(X, theta = 1.0, axis = None):
    """
    logsumexp trick
    
    Compute the softmax of each element along an axis of X.

    Parameters
    ----------
    X: ND-Array. Probably should be floats.
    theta (optional): float parameter, used as a multiplier
        prior to exponentiation. Default = 1.0
    axis (optional): axis to compute values along. Default is the
        first non-singleton axis.

    Returns an array the same size as X. The result will sum to 1
    along the specified axis.
    """

    # make X at least 2d
    y = np.atleast_2d(X)

    # find axis
    if axis is None:
        axis = next(j[0] for j in enumerate(y.shape) if j[1] > 1)

    # multiply y against the theta parameter,
    y = y * float(theta)

    # subtract the max for numerical stability
    y = y - np.expand_dims(np.max(y, axis = axis), axis)

    # exponentiate y
    y = np.exp(y)

    # take the sum along the specified axis
    ax_sum = np.expand_dims(np.sum(y, axis = axis), axis)

    # finally: divide elementwise
    p = y / ax_sum

    # flatten if X was 1D
    if len(X.shape) == 1: p = p.flatten()

    return p

In [ ]:
from sklearn.neighbors.kde import KernelDensity
import numpy as np
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
kde = KernelDensity(kernel = 'gaussian', bandwidth = 0.2).fit(X)
kde_score = kde.score_samples(X)

import scipy as sp

print(kde_score)
print(softmax_stable(kde_score, theta = 1.0, axis = None))


# ----- Contribution uncertainty

In [ ]:
import numpy as np
from scipy.stats import truncnorm
from scipy.optimize import fmin_slsqp

def func(p, r, xa, xb):
    return truncnorm.nnlf(p, r)

def constraint(p, r, xa, xb):
    a, b, loc, scale = p
    return np.array([a*scale + loc - xa, b*scale + loc - xb])

real_l, real_r = 30, 250

# --- ground truth data

loc = 50
scale = 75

# normalized boundary
a = (real_l - loc)/scale
b = (real_r - loc)/scale

# Generate some data to work with.
r = truncnorm.rvs(a, 
                  b, 
                  loc = loc, 
                  scale = scale, 
                  size = 10000)
# --- fit

loc_guess = 30
scale_guess = 90

l_guess = (real_l - loc_guess)/scale_guess
r_guess = (real_r - loc_guess)/scale_guess

par = fmin_slsqp(func, 
                 [l_guess, r_guess, loc_guess, scale_guess], 
                 f_eqcons = constraint, 
                 args = (r, real_l, real_r),
                 iprint = False, 
                 iter = 1000)
print(p0)
print(par)

import matplotlib.pyplot as plt

xmin = 0
xmax = 300
x = np.linspace(xmin, xmax, 1000)

fig, ax = plt.subplots(1, 1)
ax.plot(x, truncnorm.pdf(x, a, b, loc=loc, scale=scale),
        'r-', lw=3, alpha=0.4, label='truncnorm pdf')
ax.plot(x, truncnorm.pdf(x, *par),
        'k--', lw=1, alpha=1.0, label='truncnorm fit')
ax.hist(r, bins=15, density=True, histtype='stepfilled', alpha=0.3)
ax.legend(shadow=True)
plt.xlim(xmin, xmax)
plt.grid(True)

plt.show()

from scipy import stats
import numpy as np

# scale - exp μ, where μ is the mean of the log of the variate. (When fitting, typically you'd use the sample mean of the log of the data.)
# shape - the standard deviation of the log of the variate.

stats.lognorm(0.5, scale = np.exp(2), ).ppf(0.005)


In [ ]:
def data_reshape(data,
                 bool_target_seperate):
    # S: source
    # N: instance
    # T: time steps
    # D: dimensionality at each step
    # data: [yi, ti, [xi_src1, xi_src2, ...]]
    # by default, the first element in the xi_src1 is the auto-regressive target
    
    src_num = len(data[0][2])
    tmpx = []
    
    if bool_target_seperate == True:
        tmpx.append(np.asarray([tmp[2][0][:, 1:] for tmp in data]))
        print(np.shape(tmpx[-1]))
    
        for src_idx in range(1, src_num):
            tmpx.append(np.asarray([tmp[2][src_idx] for tmp in data]))
            print(np.shape(tmpx[-1]))
            
        tmpx.append(np.asarray([tmp[2][0][:, 0:1] for tmp in data]))
        print(np.shape(tmpx[-1]))
    else:
        for src_idx in range(src_num):
            tmpx.append(np.asarray([tmp[2][src_idx] for tmp in data]))
            print("src " + str(src_idx) + " : ", np.shape(tmpx[-1]))
    
    tmpy = np.asarray([tmp[0] for tmp in data])
    
    # output shape: x [S N T D],  y [N 1]
    return tmpx, np.expand_dims(tmpy, -1)

def mape(y, 
         yhat):
    tmp_list = []
    for idx, val in enumerate(y):
        if abs(val) > 1e-5:
            tmp_list.append(abs(1.0*(yhat[idx]-val)/val))
    return np.mean(tmp_list)

def mae(y,
        yhat):
    return np.mean(np.abs(np.asarray(y) - np.asarray(yhat)))
    
def rmse(y,
         yhat):
    return np.sqrt(np.mean((np.asarray(y) - np.asarray(yhat))**2))

def rmse_mae_quantiles(target,
                       pred,
                       q):
    '''
    Argu.:
      q: number of quantiles
    '''
    x_sort = sorted(list(target))
    
    quant_val = [x_sort[i*int(len(x_sort)/q)] for i in range(1, q)]
    quant_val.append(max(x_sort)+1)

    sq_error_quant = [0 for _ in quant_val]
    abs_error_quant = [0 for _ in quant_val]
    abs_per_error_quant = [0 for _ in quant_val]
    
    num_quant = [0 for x in quant_val]
    
    num_mape_quant = [0 for x in quant_val]
    
    for i in range(1, len(x_sort)):
        
        tmp_vol = target[i]
        tmp_resi = target[i] - pred[i]
        
        q_idx = list(map(lambda j: j>tmp_vol, quant_val)).index(True)   
        
        sq_error_quant[q_idx] += tmp_resi**2
        abs_error_quant[q_idx] += abs(tmp_resi)
        num_quant[q_idx]+=1
        
        if abs(tmp_vol)>1e-5:
            
            abs_per_error_quant[q_idx] += abs(1.0*tmp_resi/tmp_vol)
            num_mape_quant[q_idx] += 1
        
    rmse_quantiles = [ np.asscalar(np.squeeze(sqrt(sq_error_quant[j]/num_quant[j]))) for j in range(0, len(quant_val))] 
    mae_quantiles = [ np.asscalar(np.squeeze(abs_error_quant[j]/num_quant[j])) for j in range(0, len(quant_val))]
    mape_quantiles = [ np.asscalar(np.squeeze(abs_per_error_quant[j]/num_mape_quant[j])) for j in range(0, len(quant_val))] 
    
    print("cross check RMSE: ", np.sqrt(np.mean((np.squeeze(target) - pred)**2)), mae(np.squeeze(target), pred), mape(np.squeeze(target), pred))
    
    print("\n quantile rmse: ", rmse_quantiles)
    print("\n quantile mae: ",  mae_quantiles)
    print("\n quantile mape: ", mape_quantiles)
    print("\n quantiles: ", quant_val)
    
    return rmse_quantiles, mae_quantiles, mape_quantiles, quant_val

def rel_rmse_mae_quantiles(target,
                           pred,
                           q=4):
    #not symmetric!!!
    #q = number of quantiles

    x_sort = sorted(list(target))

    quant_val = [x_sort[i*int(len(x_sort)/q)] for i in range(1,q)]
    quant_val.append(max(x_sort)+1)

    sq_error_quantiles = [0 for x in quant_val]
    error_num = [0 for x in quant_val]
    abs_error_quantiles = [0 for x in quant_val]

    for i in range(1, len(x_sort)):
        tmp_vol = target[i]
        tmp_rel_resi = (tmp_vol - pred[i]) / tmp_vol
        
        q_idx = list(map(lambda j: j>tmp_vol, quant_val)).index(True)
        
        sq_error_quantiles[q_idx] += tmp_rel_resi**2
        abs_error_quantiles[q_idx] += abs(tmp_rel_resi)
        error_num[q_idx]+=1

    rmse_quantiles = [math.sqrt(sq_error_quantiles[j]/error_num[j]) for j in range(0, len(quant_val))]
    mae_quantiles = [abs_error_quantiles[j]/error_num[j] for j in range(0, len(quant_val))]
    
    print("\n quantile rel-rmse: ", rmse_quantiles)
    print("\n quantile rel-mae: ",  mae_quantiles)
    print("\n quantiles: ", quant_val)
    
    return rmse_quantiles, mae_quantiles


# ----- Results Analysis

In [ ]:

# def plot(py,
#          y,
#          plot_l,
#          plot_r,
#          path_fig,
#          src_dict):
#     # 4:"commom pattern"
#     # [bayes_mean, bayes_total_var, bayes_vola, bayes_unc, bayes_gate_src, bayes_gate_src_var, g_src_sample]
    
#     mean = py[0]
#     var = py[1]
#     var_data = np.sqrt(py[2])
#     var_model = np.sqrt(py[3])
#     gate = py[4]
#     gate_std = np.sqrt(py[5])
    
#     # [A B S]
#     gate_src_sample = np.asarray(py[6])
#     mean_src_sample = np.asarray(py[7])
#     var_src_sample  = np.asarray(py[8])
    
#     mean_low = np.maximum(0.0, mean-2.0*np.sqrt(var))
#     mean_up  = mean + 2.0*np.sqrt(var)
    
#     # -- source-wise
    
#     mean_src = np.mean(mean_src_sample, 0)
#     tmp_var_plus_sq_mean_src = var_src_sample + mean_src_sample**2
#     var_src = np.mean(tmp_var_plus_sq_mean_src, 0) - mean_src**2
    
#     mean_low_src = np.maximum(0.0, mean_src-2.0*np.sqrt(var_src))
#     mean_up_src  = mean_src + 2.0*np.sqrt(var_src)
        
#     num_src = np.shape(gate)[1]
    
#     # -- cross check
    
#     y = np.asarray(np.squeeze(y))

#     print("cross check RMSE: ", np.sqrt(np.mean((y - mean)**2)), mae(y, mean), mape(y, mean))
    
#     in_inter_sum = 0.0
#     in_cnt = 0
#     all_inter_sum = 0.0
#     all_cnt = len(y)
    
#     for idx, tmpy in enumerate(y.tolist()):
        
#         if mean_low[idx] <= tmpy and tmpy <= mean_up[idx]:
#             in_cnt += 1
#             in_inter_sum += (mean_up[idx] - mean_low[idx])
            
#         all_inter_sum += (mean_up[idx] - mean_low[idx])
            
#     print("Interval coverage ", 1.0*in_cnt/len(y))
#     print("all Interval width normalized by total number: ", 1.0*all_inter_sum/all_cnt)
#     print("all Interval width normalized by coverage number: ", 1.0*all_inter_sum/in_cnt)
#     print("within Interval width normalized by coverage number: ", in_inter_sum/in_cnt)
    
#     # -- mean and uncertainty
    
#     fig, ax = plt.subplots(figsize=(15,3));
    
#     ax.plot(range(plot_l, plot_r), y[plot_l: plot_r], label = 'truth', marker='o', color = 'k', alpha = 0.4);
#     ax.plot(range(plot_l, plot_r), mean[plot_l: plot_r], label = 'prediction', marker='o', color = 'b', alpha = 0.4);
#     ax.fill_between(range(plot_l, plot_r), mean_low[plot_l: plot_r], mean_up[plot_l: plot_r], 
#                     color = '#539caf', alpha = 0.4, label = '95.54% CI')
#     #     ax_twin = ax.twinx()
#     #     ax_twin.plot(range(plot_l, plot_r), vol[plot_l: plot_r], label = 'volatility',  marker='o', alpha=.3, color = 'g');
    
#     #ax2.tick_params(axis='y', labelcolor=color)
#     #ax.plot(range(plot_l, plot_r), vol[plot_l: plot_r], label = 'volatility',  marker='o', alpha=.3, color = 'g');
    
#     ax.set_ylim(0, 1000)
#     ax.set_title("Predicted means and uncertainties");
#     ax.set_xlabel("Time");
#     ax.set_ylabel("Value");
#     ax.legend();
    
#     ax.set_rasterized(True)
#     # fig.savefig(path_fig + '_pred.eps', format = 'eps', bbox_inches = 'tight', dpi = 300)
#     # fig.savefig(path_fig + '_pred.pdf', format = 'pdf', bbox_inches = 'tight', dpi = 100)
    
#     #  -- contribution of different sources with confidence interval
    
#     fig, ax = plt.subplots(figsize=(15, 3));
    
#     for tmp_src in range(num_src):
#         tmp_y = [i[tmp_src] for i in gate[plot_l: plot_r]]
#         tmp_yerr = [2*i[tmp_src] for i in gate_std[plot_l: plot_r]]
#         ax.errorbar(range(plot_l, plot_r), tmp_y, yerr = tmp_yerr, marker='o', label = src_dict[tmp_src], capsize = 3);
    
#     ax.legend();
#     ax.set_xlabel("Time");
#     ax.set_ylabel("Value");
#     ax.set_title("Contribution score and uncertainty of each source");
    
#     ax.set_rasterized(True)
#     fig.savefig(path_fig + '_src.pdf', format = 'pdf', bbox_inches = 'tight', dpi = 100)
    
#     # -- prediction interval of different sources
    
#     colors = ['b', 'r', 'y', 'g']
    
#     tmp_src = 0
#     fig, ax = plt.subplots(figsize = (15, 3));
#     ax.plot(range(plot_l, plot_r), y[plot_l: plot_r], label = 'truth', marker='o', color = 'k', alpha = 0.4);
#     tmp_mean = [i[tmp_src] for i in mean_src]
#     tmp_up = [i[tmp_src] for i in mean_up_src]
#     tmp_low = [i[tmp_src] for i in mean_low_src]
#     ax.plot(range(plot_l, plot_r), tmp_mean[plot_l: plot_r], label = 'prediction', marker='o', color = 'b', alpha = 0.4); 
#     ax.fill_between(range(plot_l, plot_r), tmp_low[plot_l: plot_r], tmp_up[plot_l: plot_r], color = '#539caf', alpha = 0.4, label = '')
#     ax.legend();
#     ax.set_xlabel("Time");
#     ax.set_ylabel("Value");
#     ax.set_title("");
#     ax.set_rasterized(True)
    
#     tmp_src = 1
#     fig, ax = plt.subplots(figsize = (15, 3));
#     ax.plot(range(plot_l, plot_r), y[plot_l: plot_r], label = 'truth', marker='o', color = 'k', alpha = 0.4);    
#     tmp_mean = [i[tmp_src] for i in mean_src]
#     tmp_up = [i[tmp_src] for i in mean_up_src]
#     tmp_low = [i[tmp_src] for i in mean_low_src]
#     ax.plot(range(plot_l, plot_r), tmp_mean[plot_l: plot_r], label = 'prediction', marker='o', color = 'b', alpha = 0.4); 
#     ax.fill_between(range(plot_l, plot_r), tmp_low[plot_l: plot_r], tmp_up[plot_l: plot_r], color = '#539caf', alpha = 0.4, label = '')
#     ax.legend();
#     ax.set_xlabel("Time");
#     ax.set_ylabel("Value");
#     ax.set_title("");
#     ax.set_rasterized(True)
    
#     tmp_src = 2
#     fig, ax = plt.subplots(figsize = (15, 3));
#     ax.plot(range(plot_l, plot_r), y[plot_l: plot_r], label = 'truth', marker='o', color = 'k', alpha = 0.4);
#     tmp_mean = [i[tmp_src] for i in mean_src]
#     tmp_up = [i[tmp_src] for i in mean_up_src]
#     tmp_low = [i[tmp_src] for i in mean_low_src]
#     ax.plot(range(plot_l, plot_r), tmp_mean[plot_l: plot_r], label = 'prediction', marker='o', color = 'b', alpha = 0.4); 
#     ax.fill_between(range(plot_l, plot_r), tmp_low[plot_l: plot_r], tmp_up[plot_l: plot_r], color = '#539caf', alpha = 0.4, label = '')
#     ax.legend();
#     ax.set_xlabel("Time");
#     ax.set_ylabel("Value");
#     ax.set_title("");
#     ax.set_rasterized(True)
    
#     tmp_src = 3
#     fig, ax = plt.subplots(figsize = (15, 3));
#     ax.plot(range(plot_l, plot_r), y[plot_l: plot_r], label = 'truth', marker='o', color = 'k', alpha = 0.4);
#     tmp_mean = [i[tmp_src] for i in mean_src]
#     tmp_up = [i[tmp_src] for i in mean_up_src]
#     tmp_low = [i[tmp_src] for i in mean_low_src]
#     ax.plot(range(plot_l, plot_r), tmp_mean[plot_l: plot_r], label = 'prediction', marker='o', color = 'b', alpha = 0.4); 
#     ax.fill_between(range(plot_l, plot_r), tmp_low[plot_l: plot_r], tmp_up[plot_l: plot_r], color = '#539caf', alpha = 0.4, label = '')
#     ax.legend();
#     ax.set_xlabel("Time");
#     ax.set_ylabel("Value");
#     ax.set_title("");
#     ax.set_rasterized(True)
#     #     fig.savefig(path_fig + '_src.pdf', format = 'pdf', bbox_inches = 'tight', dpi = 100)


In [ ]:
import seaborn as sns
sns.set(style="whitegrid")

def data_source_switch(gate,
                  gate_std,
                  gate_src_sample, 
                  mean_src_sample, 
                  var_src_sample):
    
    num_ins = len(gate_src_sample[0])
    num_samp = len(gate_src_sample)
    
    for i in range(num_ins):
        tmp = gate[i]
        tmp0 = tmp[0]
        tmp[0] = tmp[1]
        tmp[1] = tmp0
        tmp2 = tmp[2]
        tmp[2] = tmp[3]
        tmp[3] = tmp2
        
        tmp = gate_std[i]
        tmp0 = tmp[0]
        tmp[0] = tmp[1]
        tmp[1] = tmp0
        tmp2 = tmp[2]
        tmp[2] = tmp[3]
        tmp[3] = tmp2
            
    for j in range(num_samp):
        for i in range(num_ins):
            tmp = gate_src_sample[j][i]
            tmp0 = tmp[0]
            tmp[0] = tmp[1]
            tmp[1] = tmp0
            tmp2 = tmp[2]
            tmp[2] = tmp[3]
            tmp[3] = tmp2
        
            tmp = mean_src_sample[j][i]
            tmp0 = tmp[0]
            tmp[0] = tmp[1]
            tmp[1] = tmp0
            tmp2 = tmp[2]
            tmp[2] = tmp[3]
            tmp[3] = tmp2
            
            tmp = var_src_sample[j][i]
            tmp0 = tmp[0]
            tmp[0] = tmp[1]
            tmp[1] = tmp0
            tmp2 = tmp[2]
            tmp[2] = tmp[3]
            tmp[3] = tmp2

def plot_one(py,
         y,
         plot_l,
         plot_r,
         path_fig,
         src_dict,
         yrange, 
         market_id):
    # 4:"commom pattern"
    # [bayes_mean, bayes_total_var, bayes_vola, bayes_unc, bayes_gate_src, bayes_gate_src_var, g_src_sample]
    
    mean = py[0]
    var = py[1]
    var_data = np.sqrt(py[2])
    var_model = np.sqrt(py[3])
    # [B S]
    gate = py[4]
    gate_std = np.sqrt(py[5])
    # [A B S]
    gate_src_sample = py[6]
    mean_src_sample = py[7]
    var_src_sample  = py[8]
    
    if market_id == 2:
        data_source_switch(gate, 
                           gate_std, 
                           gate_src_sample, 
                           mean_src_sample, 
                           var_src_sample)
        
    mean_low = np.maximum(0.0, mean - 2.0*np.sqrt(var))
    mean_up  = np.minimum(800, mean + 2.0*np.sqrt(var))
    
    # -- source-wise
    
    gate_src_sample = np.asarray(gate_src_sample)
    mean_src_sample = np.asarray(mean_src_sample)
    var_src_sample  = np.asarray(var_src_sample)
    
    mean_src = np.mean(mean_src_sample, 0)
    tmp_var_plus_sq_mean_src = var_src_sample + mean_src_sample**2
    var_src = np.mean(tmp_var_plus_sq_mean_src, 0) - mean_src**2
    
    mean_low_src = np.maximum(0.0, mean_src-2.0*np.sqrt(var_src))
    mean_up_src  = mean_src + 2.0*np.sqrt(var_src)
    
    num_src = np.shape(gate)[1]
    
    # -- cross check
    
    y = np.asarray(np.squeeze(y))

    print("cross check RMSE: ", np.sqrt(np.mean((y - mean)**2)), mae(y, mean), mape(y, mean))
    
    in_inter_sum = 0.0
    in_cnt = 0
    all_inter_sum = 0.0
    all_cnt = len(y)
    
    for idx, tmpy in enumerate(y.tolist()):
        
        if mean_low[idx] <= tmpy and tmpy <= mean_up[idx]:
            in_cnt += 1
            in_inter_sum += (mean_up[idx] - mean_low[idx])
            
        all_inter_sum += (mean_up[idx] - mean_low[idx])
            
    print("Interval coverage ", 1.0*in_cnt/len(y))
    print("all Interval width normalized by total number: ", 1.0*all_inter_sum/all_cnt)
    print("all Interval width normalized by coverage number: ", 1.0*all_inter_sum/in_cnt)
    print("within Interval width normalized by coverage number: ", in_inter_sum/in_cnt)
    
    # -- mean and uncertainty
    
    fig, ax = plt.subplots(figsize=(25, 40), nrows = 6);
    
    ax[0].plot(range(plot_l, plot_r), y[plot_l: plot_r], label = 'truth', marker='o', color = 'k', alpha = 0.4,  markersize = 10);
    ax[0].plot(range(plot_l, plot_r), mean[plot_l: plot_r], label = 'prediction', marker='o', color = 'b', alpha = 0.4,  markersize = 10);
    ax[0].fill_between(range(plot_l, plot_r), mean_low[plot_l: plot_r], mean_up[plot_l: plot_r], color = '#539caf', alpha = 0.4, label = 'uncertainty')
    #     ax_twin = ax.twinx()
    #     ax_twin.plot(range(plot_l, plot_r), vol[plot_l: plot_r], label = 'volatility',  marker='o', alpha=.3, color = 'g');
    #ax2.tick_params(axis='y', labelcolor=color)
    #ax.plot(range(plot_l, plot_r), vol[plot_l: plot_r], label = 'volatility',  marker='o', alpha=.3, color = 'g');
    
    ax[0].set_ylim(yrange[0], yrange[1])
    # ax[0].set_title(); Predicted means and uncertainties
    ax[0].set_xlabel('(a)', fontsize = 30);
    ax[0].set_ylabel("Overall Prediction", fontsize = 25);
    ax[0].legend(fontsize = 25);
    ax[0].set_rasterized(True)
    ax[0].xaxis.set_tick_params(labelsize = 25)
    ax[0].yaxis.set_tick_params(labelsize = 25)
    ax[0].set_xticks(      list(range(plot_l, plot_r, 10)) )
    ax[0].set_xticklabels( list(range(0, plot_r-plot_l, 10)) )
    
    #  -- contribution score
    
    colors = ['b', 'r', 'y', 'darkviolet']
    
    for tmp_src in range(num_src):
        tmp_y = [i[tmp_src] for i in gate[plot_l: plot_r]]
        tmp_yerr = [2*i[tmp_src] for i in gate_std[plot_l: plot_r]]
        ax[1].errorbar(range(plot_l, plot_r), tmp_y, yerr = tmp_yerr, marker='o', label = src_dict[tmp_src], capsize = 3, color = colors[tmp_src], alpha = 0.4,  markersize = 10);
    
    ax[1].legend(fontsize = 25);
    ax[1].set_xlabel("(b)", fontsize = 30);
    ax[1].set_ylabel("Contribution Score", fontsize = 25);
    #ax[1].set_title("Contribution score of each source"); Contribution score of each source
    ax[1].set_rasterized(True)
    ax[1].xaxis.set_tick_params(labelsize = 25)
    ax[1].yaxis.set_tick_params(labelsize = 25)
    ax[1].set_xticks(      list(range(plot_l, plot_r, 10)) )
    ax[1].set_xticklabels( list(range(0, plot_r-plot_l, 10)) )
    
    # -- prediction interval of different sources
    
    tmp_src = 0
    ax[2].plot(range(plot_l, plot_r), y[plot_l: plot_r], label = 'truth', marker='o', color = 'k', alpha = 0.4, markersize = 10);
    tmp_mean = [i[tmp_src] for i in mean_src]
    tmp_up = [i[tmp_src] for i in mean_up_src]
    tmp_low = [i[tmp_src] for i in mean_low_src]
    ax[2].plot(range(plot_l, plot_r), tmp_mean[plot_l: plot_r], label = 'prediction by ' + src_dict[tmp_src], marker='o', color = colors[tmp_src], alpha = 0.4, markersize = 10);
    ax[2].fill_between(range(plot_l, plot_r), tmp_low[plot_l: plot_r], tmp_up[plot_l: plot_r], color = colors[tmp_src], alpha = 0.4); 
    ax[2].legend(fontsize = 25);
    ax[2].set_xlabel("(c)", fontsize = 30);
    ax[2].set_ylabel("Prediction by One Data Source", fontsize = 25);
    #     ax[2].set_title("");
    ax[2].set_rasterized(True)
    ax[2].set_ylim(yrange[0], yrange[1])
    ax[2].xaxis.set_tick_params(labelsize = 25)
    ax[2].yaxis.set_tick_params(labelsize = 25)
    ax[2].set_xticks(      list(range(plot_l, plot_r, 10)) )
    ax[2].set_xticklabels( list(range(0, plot_r-plot_l, 10)) )
    
    tmp_src = 1
    ax[3].plot(range(plot_l, plot_r), y[plot_l: plot_r], label = 'truth', marker='o', color = 'k', alpha = 0.4, markersize = 10);   
    tmp_mean = [i[tmp_src] for i in mean_src]
    tmp_up = [i[tmp_src] for i in mean_up_src]
    tmp_low = [i[tmp_src] for i in mean_low_src]
    ax[3].plot(range(plot_l, plot_r), tmp_mean[plot_l: plot_r], label = 'prediction by ' + src_dict[tmp_src], marker='o', color = colors[tmp_src], alpha = 0.4, markersize = 10); 
    ax[3].fill_between(range(plot_l, plot_r), tmp_low[plot_l: plot_r], tmp_up[plot_l: plot_r], label = '', color = colors[tmp_src], alpha = 0.4);  
    ax[3].legend(fontsize = 25);
    ax[3].set_xlabel("(d)", fontsize = 30);
    ax[3].set_ylabel("Prediction by One Data Source", fontsize = 25);
    #     ax[3].set_title("");
    ax[3].set_rasterized(True)
    ax[3].set_ylim(yrange[0], yrange[1])
    ax[3].xaxis.set_tick_params(labelsize = 25)
    ax[3].yaxis.set_tick_params(labelsize = 25)
    ax[3].set_xticks(      list(range(plot_l, plot_r, 10)) )
    ax[3].set_xticklabels( list(range(0, plot_r-plot_l, 10)) )
    
    tmp_src = 2
    ax[4].plot(range(plot_l, plot_r), y[plot_l: plot_r], label = 'truth', marker='o', color = 'k', alpha = 0.4, markersize = 10);
    tmp_mean = [i[tmp_src] for i in mean_src]
    tmp_up = [i[tmp_src] for i in mean_up_src]
    tmp_low = [i[tmp_src] for i in mean_low_src]
    ax[4].plot(range(plot_l, plot_r), tmp_mean[plot_l: plot_r], label = 'prediction by ' + src_dict[tmp_src], marker='o', color = colors[tmp_src], alpha = 0.4, markersize = 10); 
    ax[4].fill_between(range(plot_l, plot_r), tmp_low[plot_l: plot_r], tmp_up[plot_l: plot_r], color = colors[tmp_src], alpha = 0.4); 
    ax[4].legend(fontsize = 25);
    ax[4].set_xlabel("(e)", fontsize = 30);
    ax[4].set_ylabel("Prediction by One Data Source", fontsize = 25);
    #     ax[4].set_title("");
    ax[4].set_rasterized(True)
    ax[4].set_ylim(yrange[0], yrange[1])
    ax[4].xaxis.set_tick_params(labelsize = 25)
    ax[4].yaxis.set_tick_params(labelsize = 25)
    ax[4].set_xticks(      list(range(plot_l, plot_r, 10)) )
    ax[4].set_xticklabels( list(range(0, plot_r-plot_l, 10)) )
    
    tmp_src = 3
    ax[5].plot(range(plot_l, plot_r), y[plot_l: plot_r], label = 'truth', marker='o', color = 'k', alpha = 0.4, markersize = 10);
    tmp_mean = [i[tmp_src] for i in mean_src]
    tmp_up = [i[tmp_src] for i in mean_up_src]
    tmp_low = [i[tmp_src] for i in mean_low_src]
    ax[5].plot(range(plot_l, plot_r), tmp_mean[plot_l: plot_r], label = 'prediction by ' + src_dict[tmp_src], marker='o', color = colors[tmp_src], alpha = 0.4, markersize = 10);
    ax[5].fill_between(range(plot_l, plot_r), tmp_low[plot_l: plot_r], tmp_up[plot_l: plot_r], color = colors[tmp_src], alpha = 0.4); 
    ax[5].legend(fontsize = 25);
    ax[5].set_xlabel("(f)", fontsize = 30);
    ax[5].set_ylabel("Prediction by One Data Source", fontsize = 25);
    #     ax[5].set_title("");
    ax[5].set_rasterized(True)
    ax[5].set_ylim(yrange[0], yrange[1])
    ax[5].xaxis.set_tick_params(labelsize = 25)
    ax[5].yaxis.set_tick_params(labelsize = 25)
    ax[5].set_xticks(      list(range(plot_l, plot_r, 10)) )
    ax[5].set_xticklabels( list(range(0, plot_r-plot_l, 10)) )
    
#     fig.savefig(path_fig + '.pdf', format = 'pdf', bbox_inches = 'tight', dpi = 300)
#     fig.savefig(path_fig + '.eps', format = 'eps', bbox_inches = 'tight', dpi = 300)
    fig.savefig(path_fig + '.svg', format = 'svg', bbox_inches = 'tight', dpi = 50)


In [ ]:
# --- Market 1 target 1min

# ground truth data
dataset = 'market1_tar1_len10'
path_data = "../datasets/bitcoin/market1_tar1_len10/"
ts_dta = pickle.load(open(path_data + 'test.p', "rb"),
                     encoding = 'latin1')
ts_x, ts_y = data_reshape(ts_dta,
                          bool_target_seperate = True)
# prediction
path_res = "../results/m1_t1_1/"
# py: [bayes_mean, bayes_total_var, bayes_vola, bayes_unc, bayes_gate_src, bayes_gate_src_var, g_src_sample]
py = pickle.load(open(path_res + "py_" + dataset + "_top_one" + ".p", "rb"), 
                 encoding='latin1')

# src_dict = {0:"Bitfinex trans.", 
#             1:"Bitstamp trans.", 
#             2:"Bitfinex order book", 
#             3:"Bitstamp order book", 
#             4:"commom pattern"}
# plot_one(py,
#          y = ts_y,
#          plot_l = 15450,
#          plot_r = 15600,
#          path_fig = "../figure_volume/m1_t1",
#          src_dict = src_dict,
#          yrange = [0, 400], 
#          market_id = 1)

_ = rmse_mae_quantiles(target = np.squeeze(ts_y), 
                   pred = py[0], 
                   q = 4)

_ = rel_rmse_mae_quantiles(target = np.squeeze(ts_y), 
                       pred = py[0],
                       q=4)


In [ ]:
# --- Market 2 target 1min

# ground truth data
dataset = 'market2_tar1_len10'
path_data = "../datasets/bitcoin/market2_tar1_len10/"
ts_dta = pickle.load(open(path_data + 'test.p', "rb"),
                     encoding = 'latin1')
ts_x, ts_y = data_reshape(ts_dta,
                          bool_target_seperate = True)
# prediction
path_res = "../results/m2_t1_1/"
# py: [bayes_mean, bayes_total_var, bayes_vola, bayes_unc, bayes_gate_src, bayes_gate_src_var, g_src_sample]
py = pickle.load(open(path_res + "py_" + dataset + "_top_one" + ".p", "rb"), 
                 encoding='latin1')

# src_dict = {0:"Bitfinex trans.", 
#             1:"Bitstamp trans.", 
#             2:"Bitfinex order book", 
#             3:"Bitstamp order book", 
#             4:"commom pattern"}
# plot_one(py,
#          y = ts_y,
#          plot_l = 29150,
#          plot_r = 29300, 
#          path_fig = "../figure_volume/m2_t1", 
#          src_dict = src_dict, 
#          yrange = [0, 100], 
#          market_id = 2)

rmse_mae_quantiles(target = ts_y, pred = py[0], q = 4)

rel_rmse_mae_quantiles(target = np.squeeze(ts_y), 
                       pred = py[0],
                       q=4)


In [ ]:
# --- Market 1 target 5min

# ground truth data
dataset = 'market1_tar5_len10'
path_data = "../datasets/bitcoin/market1_tar5_len10/"
ts_dta = pickle.load(open(path_data + 'test.p', "rb"),
                     encoding = 'latin1')
ts_x, ts_y = data_reshape(ts_dta,
                          bool_target_seperate = True)
# prediction
path_res = "../results/m1_t5_1/"
# py: [bayes_mean, bayes_total_var, bayes_vola, bayes_unc, bayes_gate_src, bayes_gate_src_var, g_src_sample]
py = pickle.load(open(path_res + "py_" + dataset + "_bayes_one" + ".p", "rb"), 
                 encoding='latin1')

# src_dict = {0:"Bitfinex trans.",
#             1:"Bitstamp trans.", 
#             2:"Bitfinex order book",
#             3:"Bitstamp order book", 
#             4:"commom pattern"}
# plot_one(py,
#          y = ts_y,
#          plot_l = 800,
#          plot_r = 950,
#          path_fig = "../figure_volume/m1_t5",
#          src_dict = src_dict,
#          yrange = [0, 500], 
#          market_id = 1)

rmse_mae_quantiles(target = ts_y, pred = py[0], q = 4)

rel_rmse_mae_quantiles(target = np.squeeze(ts_y), 
                       pred = py[0],
                       q=4)


In [ ]:
# --- Market 2 target 5min

# ground truth data
dataset = 'market2_tar5_len10'
path_data = "../datasets/bitcoin/market2_tar5_len10/"
ts_dta = pickle.load(open(path_data + 'test.p', "rb"),
                     encoding = 'latin1')
ts_x, ts_y = data_reshape(ts_dta,
                          bool_target_seperate = True)
# prediction
path_res = "../results/m2_t5_1/"
# py: [bayes_mean, bayes_total_var, bayes_vola, bayes_unc, bayes_gate_src, bayes_gate_src_var, g_src_sample]
py = pickle.load(open(path_res + "py_" + dataset + "_bayes_one" + ".p", "rb"), 
                 encoding='latin1')

# src_dict = {0:"Bitfinex trans.", 
#             1:"Bitstamp trans.", 
#             2:"Bitfinex order book", 
#             3:"Bitstamp order book", 
#             4:"commom pattern"}
# plot_one(py,
#          y = ts_y,
#          plot_l = 50,
#          plot_r = 200, 
#          path_fig = "../figure_volume/m2_t5", 
#          src_dict = src_dict, 
#          yrange = [0, 900], 
#          market_id = 2)

rmse_mae_quantiles(target = ts_y, pred = py[0], q = 4)

rel_rmse_mae_quantiles(target = np.squeeze(ts_y), 
                       pred = py[0],
                       q=4)


In [ ]:
# --- Market 1 target 10min

# ground truth data
dataset = 'market1_tar10_len10'
path_data = "../datasets/bitcoin/market1_tar10_len10/"
ts_dta = pickle.load(open(path_data + 'test.p', "rb"),
                     encoding = 'latin1')
ts_x, ts_y = data_reshape(ts_dta,
                          bool_target_seperate = True)
# prediction
path_res = "../results/m1_t10_1/"
# py: [bayes_mean, bayes_total_var, bayes_vola, bayes_unc, bayes_gate_src, bayes_gate_src_var, g_src_sample]
py = pickle.load(open(path_res + "py_" + dataset + "_top_one" + ".p", "rb"), 
                 encoding='latin1')

# src_dict = {0:"Bitfinex trans.", 
#             1:"Bitstamp trans.", 
#             2:"Bitfinex order book", 
#             3:"Bitstamp order book", 
#             4:"commom pattern"}
# plot_one(py,
#          y = ts_y,
#          plot_l = 1,
#          plot_r = 150,
#          path_fig = "../figure_volume/m1_t10",
#          src_dict = src_dict,
#          yrange = [0, 800], 
#          market_id = 1)

rmse_mae_quantiles(target = ts_y, pred = py[0], q = 4)

rel_rmse_mae_quantiles(target = np.squeeze(ts_y), 
                       pred = py[0],
                       q=4)


In [ ]:
# --- Market 2 target 10min

# ground truth data
dataset = 'market2_tar10_len10'
path_data = "../datasets/bitcoin/market2_tar10_len10/"
ts_dta = pickle.load(open(path_data + 'test.p', "rb"),
                     encoding = 'latin1')
ts_x, ts_y = data_reshape(ts_dta,
                          bool_target_seperate = True)
# prediction
path_res = "../results/m2_t10_1/"
# py: [bayes_mean, bayes_total_var, bayes_vola, bayes_unc, bayes_gate_src, bayes_gate_src_var, g_src_sample]
py = pickle.load(open(path_res + "py_" + dataset + "_top_one" + ".p", "rb"), 
                 encoding='latin1')

# src_dict = {0:"Bitfinex trans.", 
#             1:"Bitstamp trans.", 
#             2:"Bitfinex order book",
#             3:"Bitstamp order book", 
#             4:"commom pattern"}
# plot_one(py,
#          y = ts_y,
#          plot_l = 150,
#          plot_r = 300,
#          path_fig = "../figure_volume/m2_t10",
#          src_dict = src_dict,
#          yrange = [0, 400], 
#          market_id = 2)

rmse_mae_quantiles(target = ts_y, pred = py[0], q = 4)

rel_rmse_mae_quantiles(target = np.squeeze(ts_y), 
                       pred = py[0],
                       q=4)


In [ ]:

# ground truth data

dataset = 'market2_tar5_len10'
path_data = "../datasets/bitcoin/market2_tar5_len10/"

dta = pickle.load(open(path_data + 'train.p', "rb"),
                     encoding = 'latin1')
x, y = data_reshape(dta,
                    bool_target_seperate = True)

dta = pickle.load(open(path_data + 'val.p', "rb"),
                     encoding = 'latin1')
x, y = data_reshape(dta,
                    bool_target_seperate = True)

dta = pickle.load(open(path_data + 'test.p', "rb"),
                     encoding = 'latin1')
x, y = data_reshape(dta,
                    bool_target_seperate = True)


In [ ]:

# ground truth data

dataset = 'market2_tar1_len10'
path_data = "../datasets/bitcoin/market2_tar1_len10/"

dta = pickle.load(open(path_data + 'train.p', "rb"),
                     encoding = 'latin1')
x, y = data_reshape(dta,
                    bool_target_seperate = True)

dta = pickle.load(open(path_data + 'val.p', "rb"),
                     encoding = 'latin1')
x, y = data_reshape(dta,
                    bool_target_seperate = True)

dta = pickle.load(open(path_data + 'test.p', "rb"),
                     encoding = 'latin1')
x, y = data_reshape(dta,
                    bool_target_seperate = True)

# ----------

In [ ]:
import numpy as np
from scipy.stats import truncnorm
from scipy.optimize import fmin_slsqp

import matplotlib.pyplot as plt

def func(p, r, xa, xb):
    return truncnorm.nnlf(p, r)

def constraint(p, r, xa, xb):
    a, b, loc, scale = p
    return np.array([a*scale + loc - xa, b*scale + loc - xb])

xa, xb = 30, 250 
loc = 50
scale = 75

a = (xa - loc)/scale
b = (xb - loc)/scale

# Generate some data to work with.
r = truncnorm.rvs(a, b, loc=loc, scale=scale, size=10000)

loc_guess = 30
scale_guess = 90
a_guess = (xa - loc_guess)/scale_guess
b_guess = (xb - loc_guess)/scale_guess
p0 = [a_guess, b_guess, loc_guess, scale_guess]

par = fmin_slsqp(func, p0, f_eqcons=constraint, args=(r, xa, xb),
                 iprint=False, iter=1000)

xmin = 0
xmax = 300
x = np.linspace(xmin, xmax, 1000)

fig, ax = plt.subplots(1, 1)
ax.plot(x, truncnorm.pdf(x, a, b, loc=loc, scale=scale),
        'r-', lw=3, alpha=0.4, label='truncnorm pdf')
ax.plot(x, truncnorm.pdf(x, *par),
        'k--', lw=1, alpha=1.0, label='truncnorm fit')
ax.hist(r, bins=15, density=True, histtype='stepfilled', alpha=0.3)
ax.legend(shadow=True)
plt.xlim(xmin, xmax)
plt.grid(True)

plt.show()

In [ ]:
path_data = "../dataset/bitcoin/market1_tar5_len10/"

tr_dta = pickle.load(open(path_data + 'train.p', "rb"), encoding = 'latin1')    
tr_x, tr_y = data_reshape(tr_dta,
                          bool_target_seperate = False)

val_dta = pickle.load(open(path_data + 'val.p', "rb"), encoding = 'latin1')
val_x, val_y = data_reshape(val_dta,
                          bool_target_seperate = False)

ts_dta = pickle.load(open(path_data + 'test.p', "rb"), encoding = 'latin1')
ts_x, ts_y = data_reshape(ts_dta,
                          bool_target_seperate = False)

fig, ax = plt.subplots(nrows = 1, ncols = 3,figsize=(18,5));

ax[0].hist(tr_y, 2000, label = 'Train', density = True);
ax[0].legend()
ax[0].set_xlim([-10, 500]);
ax[0].set_ylim([0, 0.05]);

ax[1].hist(val_y, 2000, label = 'Val.', density = True);
ax[1].legend()
ax[1].set_xlim([-10, 500]);
ax[1].set_ylim([0, 0.05]);

ax[2].hist(ts_y, 2000, label = 'Test', density = True);
ax[2].legend()
ax[2].set_xlim([-10, 500]);
ax[2].set_ylim([0, 0.05]);

# ax.set_xlim([-10, 500]);
# ax.set_title("True values in training data");

# # fig, ax = plt.subplots();
# ax.hist(val_y, 2000, label = 'Val.');
# ax.set_xlim([-10, 500]);
# ax.set_title("True values in validation data");


# # fig, ax = plt.subplots();
# ax.hist(ts_y, 2000, label = 'Test');
# ax.set_xlim([-10, 500]);
# ax.set_title("True values in testing data");


In [ ]:
#  hyper_para generator 

from utils_training import *

para_hpara_range = [[0.001, 0.001], [16, 80], [1e-7, 0.1]]
para_hpara_n_trial = 15

hpara_generator = hyper_para_random_search(para_hpara_range, 
                                              para_hpara_n_trial)

tmp_hpara = hpara_generator.hpara_trial()
cnt = 0

while tmp_hpara != None:
    
    print(tmp_hpara)
    
    tmp_hpara = hpara_generator.hpara_trial()
    
    cnt += 1
    
print(cnt)

# ------ 

from utils_training import *


para_lr_range = [0.001, ]
para_batch_range = [64, 32, 16, 80]
para_l2_range = [1e-7, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1]

hpara_generator = hpara_grid_search([para_lr_range, para_batch_range, para_l2_range])

tmp_hpara = hpara_generator.hpara_trial()
cnt = 0

while tmp_hpara != None:
    
    print(tmp_hpara)
    
    tmp_hpara = hpara_generator.hpara_trial()
    
    cnt += 1
    
print(cnt)
    

In [ ]:

mixture_gaussian = (norm.pdf(x_axis, -3, 1) + norm.pdf(x_axis, 3, 1)) / 2


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

n = 10000 # number of sample to be drawn
mu = [-6, 5]
sigma = [2, 3]
samples = []

for i in range(n): # iteratively draw samples
    Z = np.random.choice([0,1]) # latent variable
    samples.append(np.random.normal(mu[Z], sigma[Z], 1))
    
sns.distplot(samples, hist=False)
plt.show()
sns.distplot(samples)
plt.show()



In [ ]:

import numpy as np
import matplotlib.pyplot as plt

distributions = [
    {"type": np.random.lognormal, "kwargs": {"mean": -0.35, "sigma": 0.2}},
]

coefficients = np.array([1.0])
coefficients /= coefficients.sum()      # in case these did not add up to 1
sample_size = 100000

num_distr = len(distributions)
data = np.zeros((sample_size, num_distr))
for idx, distr in enumerate(distributions):
    data[:, idx] = distr["type"](size=(sample_size,), **distr["kwargs"])
    
random_idx = np.random.choice(np.arange(num_distr), size = (sample_size,), p = coefficients)
sample = data[np.arange(sample_size), random_idx]
plt.hist(sample, bins=100, density=True)
plt.show()


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

distributions = [
    {"type": np.random.lognormal, "kwargs": {"mean": -0.35, "sigma": 0.2}},
    {"type": np.random.lognormal, "kwargs": {"mean": -0.1, "sigma": 0.2}},
    {"type": np.random.lognormal, "kwargs": {"mean": -0.002, "sigma": 0.1}},
]

coefficients = np.array([0.5, 0.2, 0.3])
coefficients /= coefficients.sum()      # in case these did not add up to 1
sample_size = 100000

num_distr = len(distributions)
data = np.zeros((sample_size, num_distr))
for idx, distr in enumerate(distributions):
    data[:, idx] = distr["type"](size=(sample_size,), **distr["kwargs"])
    
random_idx = np.random.choice(np.arange(num_distr), size = (sample_size,), p = coefficients)
sample = data[np.arange(sample_size), random_idx]
plt.hist(sample, bins=100, density=True)
plt.show()


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

distributions = [
    {"type": np.random.lognormal, "kwargs": {"mean": -0.35, "sigma": 0.2}},
    {"type": np.random.lognormal, "kwargs": {"mean": -0.1, "sigma": 0.2}},
    {"type": np.random.lognormal, "kwargs": {"mean": -0.002, "sigma": 0.1}},
]

coefficients = np.array([0.1, 0.6, 0.3])
coefficients /= coefficients.sum()      # in case these did not add up to 1
sample_size = 100000

num_distr = len(distributions)
data = np.zeros((sample_size, num_distr))
for idx, distr in enumerate(distributions):
    data[:, idx] = distr["type"](size=(sample_size,), **distr["kwargs"])
    
random_idx = np.random.choice(np.arange(num_distr), size = (sample_size,), p = coefficients)
sample = data[np.arange(sample_size), random_idx]
plt.hist(sample, bins=100, density=True)
plt.show()
